In [17]:
%reset -f

from groq import Groq
import os
from dotenv import load_dotenv
import tiktoken  # For token counting

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

max_tokens = 1000  # Max tokens for a single request
max_comment_len = 500  # Max comments to

client = Groq(api_key=groq_api_key)




# Sample long comments
comments = [
    "من ملك هذا حتى يلغي العيد؟ حتى مذبحش العيد مسلم ويقوم بشعائر العيد",
    "فرحانين وكأن الدولة نقصات ليهم من البطالة ومن غلاء الأسعار، ولكن فالحقيقة لا ها",
    "من الجميل أنكم قمتم بدراسة الوضع حول عيد الاضحى … ولكن عليكم أيضا دراسة وضع المواطنين الذين يعانون من ارتفاع الاسعار و الغلاء المعيشي على طول العام …",
    "شناقة صباح الخير"
]


# Process sentiment for each chunk
sentiments = {"positive": 0, "negative": 0, "neutral": 0}
total_comments = 0

for comment in comments:
    comment = comment[:500]  # Limit to 500 tokens
    prompt = f"""
    Analyze the sentiment of the following comment and provide:
    1. The percentage of positive, negative, and neutral comment.
    
    Comment:
    {comment}
    """
    
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}, {"role": "system", "content": "Please do not use markdown syntax like **bold** in your response."}],
        temperature=0.5,
        max_completion_tokens=500,
        stream=False
    )
    
    sentiment_result = response.choices[0].message.content
    print("===============================================")
    print("comment:", comment)
    print("sentiment_result:", sentiment_result)
    print("===============================================")
    # Parse sentiment result (assuming response format is structured)
    # Example response format:
    # "Positive: 60%\nNegative: 30%\nNeutral: 10%"
    for line in sentiment_result.split("\n"):
        if "Positive" in line:
            sentiments["positive"] += float(line.split(":")[1].strip().split("%")[0].strip())
        elif "Negative" in line:
            sentiments["negative"] += float(line.split(":")[1].strip().split("%")[0].strip())
        elif "Neutral" in line:
            sentiments["neutral"] += float(line.split(":")[1].strip().split("%")[0].strip())
    
    total_comments += 1

# Compute final sentiment percentages
for key in sentiments:
    sentiments[key] = round(sentiments[key] / total_comments, 2)

# Output Sentiment Results
print(f"Final Sentiment Analysis:")
print(f"Positive: {sentiments['positive']}%")
print(f"Negative: {sentiments['negative']}%")
print(f"Neutral: {sentiments['neutral']}%")


comment: من ملك هذا حتى يلغي العيد؟ حتى مذبحش العيد مسلم ويقوم بشعائر العيد
sentiment_result: The sentiment of the given comment is negative. 

Here is the breakdown of the sentiment analysis:
1. Positive: 0%
2. Negative: 100%
3. Neutral: 0%

The comment expresses dissatisfaction and frustration with someone who claims to be a king but does not observe the Eid celebrations, which is a significant Islamic holiday. The tone is disapproving and critical, indicating a strongly negative sentiment.
comment: فرحانين وكأن الدولة نقصات ليهم من البطالة ومن غلاء الأسعار، ولكن فالحقيقة لا ها
sentiment_result: The sentiment of the given comment is negative. 

Here's the breakdown:
1. Positive: 0% - There are no positive sentiments expressed in the comment.
2. Negative: 100% - The comment expresses dissatisfaction and frustration with the current state of affairs, specifically mentioning unemployment and high prices.
3. Neutral: 0% - The tone of the comment is not neutral, as it clearly conveys a ne